In [ ]:
#Install all dependency packages for the course
#Remember to execute this before running any of the exercises
!pip install tenacity==9.0.0
!pip install langchain==0.3.12
!pip install langchain-openai==0.2.12
!pip install langchain_community==0.3.12
!pip install langgraph==0.2.59
!pip install pysqlite3-binary==0.5.4
!pip install langchain_chroma==0.1.4
!pip install pandas==2.2.3
!pip install pypdf==5.1.0
!pip install nbformat==5.10.4

  Using cached langchain_chroma-0.1.4-py3-none-any.whl.metadata (1.6 kB)
  Using cached chromadb-0.5.23-py3-none-any.whl.metadata (6.8 kB)
  Using cached chroma_hnswlib-0.7.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (252 bytes)
  Using cached posthog-6.0.0-py3-none-any.whl.metadata (6.0 kB)
  Using cached onnxruntime-1.22.0-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (4.5 kB)
  Using cached opentelemetry_api-1.34.1-py3-none-any.whl.metadata (1.5 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.34.1-py3-none-any.whl.metadata (2.4 kB)
  Using cached opentelemetry_instrumentation_fastapi-0.55b1-py3-none-any.whl.metadata (2.2 kB)
  Using cached opentelemetry_sdk-1.34.1-py3-none-any.whl.metadata (1.6 kB)
  Using cached tokenizers-0.20.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached PyPika-0.48.9.tar.gz (67 kB)
  Installing build dependencies ... done
  Getting requirements to build w

## 2.3. Setup Function Tools for ReAct Agent

In [ ]:
from langchain_core.tools import tool

#Tool annotation identifies a function as a tool automatically
@tool
def find_sum(x:int, y:int) -> int :
    #The docstring comment describes the capabilities of the function
    #It is used by the agent to discover the function's inputs, outputs and capabilities
    """
    This function is used to add two numbers and return their sum.
    It takes two integers as inputs and returns an integer as output.
    """
    return x + y

@tool
def find_product(x:int, y:int) -> int :
    """
    This function is used to multiply two numbers and return their product.
    It takes two integers as inputs and returns an integer as ouput.
    """
    return x * y


## 2.4. Create a basic ReAct Agent

https://ai.azure.com/resource/deployments/%2Fsubscriptions%2Fed471d4f-f9d4-4900-b4df-f8d045bd0812%2FresourceGroups%2Fopenai%2Fproviders%2FMicrosoft.CognitiveServices%2Faccounts%2F24813-mci9rvhm-eastus2%2Fdeployments%2Fgpt-4o?wsid=/subscriptions/ed471d4f-f9d4-4900-b4df-f8d045bd0812/resourceGroups/openai/providers/Microsoft.CognitiveServices/accounts/24813-mci9rvhm-eastus2&tid=283ffb50-a30b-488c-90f4-cdae4f7ae6d1

In [ ]:
from langchain_openai import AzureChatOpenAI
import os
#Setup the LLM for the agent

#API info. Replace with your own keys and end points
os.environ["AZURE_OPENAI_API_KEY"]="<YOUR KEY>"
os.environ["AZURE_OPENAI_ENDPOINT"]="<YOUR END POINT>"

#Setup the LLM
model = AzureChatOpenAI(
    azure_deployment="gpt-4o" ,
    api_version="2024-12-01-preview",
    model="gpt-4o"
)

In [ ]:
# Test the model
# response = model.invoke("Hello, how are you?")
# print(response.content)

Hello! I'm just a bunch of code, so I don't have feelings, but I'm here and ready to help you with anything you need. How are you doing today? 😊


In [ ]:
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import AIMessage,HumanMessage,SystemMessage

#Create list of tools available to the agent
agent_tools=[find_sum, find_product]

#System prompt
system_prompt = SystemMessage(
    """You are a Math genius who can solve math problems. Solve the
    problems provided by the user, by using only tools available.
    Do not solve the problem yourself"""
)

agent_graph=create_react_agent(
    model=model,
    state_modifier=system_prompt,
    tools=agent_tools)


## 2.5. Execute the ReAct Agent

In [ ]:
#Example 1
inputs = {"messages":[("user","what is the sum of 2 and 3 ?")]}

# result = agent_graph.invoke(inputs)

# #Get the final answer
# print(f"Agent returned : {result['messages'][-1].content} \n")

# print("Step by Step execution : ")
# for message in result['messages']:
#     print(message.pretty_repr())

Agent returned : The sum of 2 and 3 is 5. 

Step by Step execution : 
================================ Human Message =================================

what is the sum of 2 and 3 ?
================================== Ai Message ==================================
Tool Calls:
  find_sum (call_hkqbee3UPIsJEfw8qMN2vlYT)
 Call ID: call_hkqbee3UPIsJEfw8qMN2vlYT
  Args:
    x: 2
    y: 3
================================= Tool Message =================================
Name: find_sum

5
================================== Ai Message ==================================

The sum of 2 and 3 is 5.


In [ ]:
#Example 2
inputs = {"messages":[("user","What is 3 multipled by 2 and 5 + 1 ?")]}

# result = agent_graph.invoke(inputs)

# #Get the final answer
# print(f"Agent returned : {result['messages'][-1].content} \n")

# print("Step by Step execution : ")
# for message in result['messages']:
#     print(message.pretty_repr())

Agent returned : The result of \( 3 \times 2 \) is \( 6 \), and \( 5 + 1 \) is also \( 6 \). 

Step by Step execution : 
================================ Human Message =================================

What is 3 multipled by 2 and 5 + 1 ?
================================== Ai Message ==================================
Tool Calls:
  find_product (call_0QYJvfeIc4fYRPFyLAlWoX29)
 Call ID: call_0QYJvfeIc4fYRPFyLAlWoX29
  Args:
    x: 3
    y: 2
  find_sum (call_WO7trwO62CbQITYa29dDMdwj)
 Call ID: call_WO7trwO62CbQITYa29dDMdwj
  Args:
    x: 5
    y: 1
================================= Tool Message =================================
Name: find_product

6
================================= Tool Message =================================
Name: find_sum

6
================================== Ai Message ==================================

The result of \( 3 \times 2 \) is \( 6 \), and \( 5 + 1 \) is also \( 6 \).


## 2.6. Debugging the Agent

In [ ]:
agent_graph=create_react_agent(
    model=model,
    state_modifier=system_prompt,
    tools=agent_tools,
    debug=True)

inputs = {"messages":[("user","what is the sum of 2 and 3 ?")]}

# result = agent_graph.invoke(inputs)

[-1:checkpoint] State at the end of step -1:
{'messages': []}
[0:tasks] Starting 1 task for step 0:
- __start__ -> {'messages': [('user', 'what is the sum of 2 and 3 ?')]}
[0:writes] Finished step 0 with writes to 1 channel:
- messages -> [('user', 'what is the sum of 2 and 3 ?')]
[0:checkpoint] State at the end of step 0:
{'messages': [HumanMessage(content='what is the sum of 2 and 3 ?', additional_kwargs={}, response_metadata={}, id='1bac64c7-385c-403d-a4dd-f7f7e35e9ca4')]}
[1:tasks] Starting 1 task for step 1:
- agent -> {'is_last_step': False,
 'messages': [HumanMessage(content='what is the sum of 2 and 3 ?', additional_kwargs={}, response_metadata={}, id='1bac64c7-385c-403d-a4dd-f7f7e35e9ca4')],
 'remaining_steps': 24}
[1:writes] Finished step 1 with writes to 1 channel:
- messages -> [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_hEgplpWIBWarjf5e1llyjjnb', 'function': {'arguments': '{"x":2,"y":3}', 'name': 'find_sum'}, 'type': 'function'}], 'refusal': None}